# Example: Checking Compute Targets 🖥️ and Registering Environments 🐳

We will follow the following steps:

1. Checking available Compute Targets in your Azure Machine Learning Workspace
2. Creating a Docker-based environment for our Machine Learning tasks
3. Registering the created environment to be used across different experiments and pipelines 
4. Checking the registered environments in your Azure Machine Learning Workspace

Let's get started!


# Compute Targets in the Azure Machine Learning Workspace 🖥️

Compute targets in Azure Machine Learning 🧪 are the resources or environments where you run your training scripts or host your service deployments. These can be local or cloud resources ☁️.

Here is a some sample Python code on how to check the available Compute Targets in your Azure Machine Learning Workspace:

# Import Libraries and Set Up Workspace 🏗️

As always, we start by importing the necessary libraries and setting up our Azure Machine Learning Workspace.

In [ ]:
from azureml.core import Workspace, Datastore
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Dataset
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential

subscription_id = #use your subscription id
resource_group = #use your resource group
workspace_name = #use your workspace name

# Log in using interactive Auth
auth = InteractiveLoginAuthentication()

# Connect to workspace
workspace = Workspace(subscription_id=subscription_id,
                      resource_group=resource_group,
                      workspace_name=workspace_name,
                      auth=auth,
                      )

# Connect to the ML Client
credential = InteractiveBrowserCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

# Inspect available compute targets 🎯


In [ ]:
# Check for existing compute targets
compute_targets = workspace.compute_targets

for name, ct in compute_targets.items():
    print(f'🖥️ {name}: {ct.type}, Provisioning state: {ct.provisioning_state}')

To use any of these compute targets in a command job or pipeline (more on these later) you just need the name of the compute target.

## Step 3: Creating a Docker-based Environment for Our Machine Learning Tasks 🐳

The environment in Azure Machine Learning is a named object that specifies the packages, environment variables, software settings, and hardware used for model training and scoring.

In our case, we'll create a Docker-based environment, and we will specify some packages that we want to use in our machine learning tasks.

Here's a Python sample on how to create a Docker-based environment:

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

# Create a new environment
env = Environment(name="my_docker_environment")

# Create a base Docker image to use for the environment
env.docker.base_dockerfile = """
FROM ubuntu:18.04
RUN apt-get update && \
    apt-get install -y python3-pip && \
    pip3 install azureml-defaults
"""

# Create a new Conda dependencies object
conda_dep = CondaDependencies.create(python_version="3.8")

# Add some package dependencies
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("scikit-learn")

# Add the dependencies to the environment
env.python.conda_dependencies = conda_dep

# Specify a Docker-based environment
docker_config = DockerConfiguration(use_docker=True)

# Register the environment to re-use later
env.register(workspace)


In this script:

- `Environment(name="my_docker_environment")` is used to create a new environment.
- `env.docker.enabled = True` and `env.python.version = "3.8"` are used to specify that we are using a Docker-based environment and set the Python version.
- `conda_dep.add_conda_package("numpy")` etc., are used to specify the packages we want to install in our environment.
- `env.register(workspace)` is used to register the environment with the Azure Machine Learning Workspace, allowing us to reuse it in the future.

There are several ways to create an environment in Azure Machine Learning. You can create an environment from:

- A Conda or pip requirements file
- A Conda environment YAML file
- A Dockerfile
- A curated list of packages

You can also create an environment from an existing environment object, or from an existing Docker image.

If you already have a Docker image that you want to use, you can create an environment from it using the following code:

```python
from azure.ai.ml.entities import Environment

# Create a custom environment.
custom_env_name = #your name here
version = "1.0.1"

env = Environment(
    name=custom_env_name,
    conda_file=#path to a condafile with dependencies not already installed in the base image,
    image= #"docker_user_name/docker_image_name:latest",
    version=version,
)

# Register the environment to the workspace.
env.register(workspace=ws)
```

# Checking the registered environments in your Azure Machine Learning Workspace ✅

In [ ]:
# List all available environments.
envs = ml_client.environments.list()
for env in envs:
    print(f"Environment name: {env.name}, latest version: {env.latest_version}")

The above code lists all the registered environments in your Azure Machine Learning Workspace. To use any of these environments in a command job or pipeline (more on these later) you just need the name of the environment.